In [1]:
#эта программа была написана на Windows; совместимость с терминалами других ОС гарантирована быть не может :(
#положите программу в папку с mystem.exe
#обратите внимание: т.к. программа генерирует текст для чтения (.html) вместо plain, строки с @ закомментированы

In [2]:
import urllib.request
import os
import shutil
from bs4 import BeautifulSoup
import re
import time



In [3]:
def download_page(pageUrl):
    try:
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')
        soup = BeautifulSoup(text, 'html.parser')
        return soup
    except Exception as err:
        print('Error', err, 'at', pageUrl)
        return
            
        

In [4]:
def catalogue(date, ftype):
    khanty = os.path.join(path, 'Khanty_yasang', ftype)
    folder = os.path.join(khanty, date[-4:], date[3:5])
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder
    
    

In [5]:
def read_mode(article, author, title, date, topic, source, thelink, path):
    paragraphs = []
    imgs = []
    for div in article.find_all('div', {'class': 'field-name-field-body-russian'}):
        for p in div.find_all('p', {'class': None}):
            paragraphs.append(p)
    for img in article.find_all('img', {'class': None}):
        if 'default_images' not in str(img) and 'yandex' not in str(img) and 'authors' not in str(img):
            if img not in imgs:
                imgs.append(img)
    folder = catalogue(date, 'plain')
    fname = os.path.join(folder, thelink[-4:]) + '.html'
    with open(fname, 'w', encoding='utf-8') as f:
        f.write('''<!--@au %s
@ti %s
@da %s
@topic %s
@url %s
-->\n''' % (author, title, date, topic, source))
        if len(imgs) > 0:
            f.write(str(imgs[0]))
        if len(paragraphs) > 0:
            for p in paragraphs:
                f.write(str(p) + '\n')
        if len(imgs) > 1:
            for img in imgs[1:]:
                f.write(str(img)+ '\n')
    return fname


In [6]:
def mystem(fname, date, thelink):
    with open(fname, encoding='utf-8') as f:
        text = f.read()
    with open('plain_text.txt', 'w', encoding='utf-8') as f:
        f.write(re.sub(r'<.*?>', '', re.sub(r'<!--.*?-->\n', '', re.sub(r'@.*?\n', '', text))))
    plain_folder = catalogue(date, 'mystem-plain')
    plain_name = thelink[-4:] + '.txt'
    os.system('mystem.exe plain_text.txt -clid --eng-gr ' + plain_name)
    shutil.move(os.path.join(path, plain_name), plain_folder)
    xml_folder = catalogue(date, 'mystem-xml')
    xml_name = thelink[-4:] + '.xml'
    os.system('mystem.exe plain_text.txt -clid --eng-gr --format xml ' + xml_name)
    shutil.move(os.path.join(path, xml_name), xml_folder)
    
    

In [7]:
def get_articles(issue_link):
    issue = download_page(base + issue_link)
    for article_link in issue.find_all('a', {'rel': 'tag'}):
        thelink = article_link.get('href')
        source = base + thelink
        article = download_page(source)
        try:
            author = article.find('div', {'class': 'view-author-article'}).get_text().strip()
        except AttributeError:
            author = 'None'
        titles = article.find_all('div', {'class': 'field-title'})
        if len(titles) > 1:
            title = titles[1].get_text().strip()
        elif len(titles) == 1:
            title = titles[0].get_text().strip()
        else:
            title = 'None'
        get_date = article.find('div', {'class': 'data'}).get_text()
        raw_date =  re.sub(r'(\d{2})/(\d{2})/(\d{4})', r'\2.\1.\3', get_date)
        date = re.findall(r'\d{2}\.\d{2}\.\d{4}', raw_date)[0]
        try:
            topic = article.find('div', {'class': 'field-name-field-rubric-khanty'}).get_text().split(' - ')[1].strip()
        except AttributeError:
            topic = 'None'
        fname = read_mode(article, author, title, date, topic, source, thelink, path)
        with open('Khanty_yasang\meta.csv', 'a', encoding='utf-8') as f:
            f.write('%s\t%s\t%s\t%s\tпублицистика\t%s\tнейтральный\tн-возраст\tн-уровень\
\tокружная\t%s\tХӑнты ясаӈ\t%s\tгазета\tРоссия\tХМАО\tru\
\n' % (fname[-38:], author, title, date, topic, source, date[-4:]))
            f.close()
        mystem(fname, date, thelink)
        
            

In [8]:
def main():
    try:
        os.mkdir('Khanty_yasang')
    except FileExistsError:
        pass
    with open('Khanty_yasang\meta.csv', 'w', encoding='utf-8') as f:
        f.write('path\tauthor\theader\tcreated\tsphere\ttopic\tstyle\taudience_age\taudience_level\
    \taudience_size\tsource\tpublication\tpubl_year\tmedium\tcountry\tregion\tlanguage\n')
        f.close()
    year = 2017
    link = download_page(url + str(year))
    for year_number in link.find_all('span', {'class': 'field-content'}):
        for number in year_number.find_all('a'):
            issue_link = number.get('href')
            get_articles(issue_link)
            time.sleep(2)
    os.remove('plain_text.txt')

In [9]:
if __name__ == '__main__':
    base = 'http://www.khanty-yasang.ru'
    url = 'http://www.khanty-yasang.ru/khanty-yasang/archive/'
    path = os.getcwd()
    main()